In [1]:
# (Optional) Enable interactive plotting in Jupyter

# For Jupyter notebooks
# %matplotlib notebook

# For Jupyter lab
%matplotlib widget

In [3]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py

cwd = os.getcwd()
pardir = os.path.abspath(os.path.join(cwd, ".."))
modeldir = os.path.abspath(os.path.join(pardir, "models"))

if pardir not in sys.path:
    sys.path.append(pardir)

from jDAS import JDAS

# Load DAS data

For this tutorial, a small sample of real-world DAS data is loaded from `sample_data.npy`. These recordings have a temporal sampling frequency of 50 Hz and a gauge length (and channel spacing) of 19.2 m. The DAS data are organised in a matrix of shape (`# of channels, # of time samples`).

**Important**: the input data need to be approximately normalised. This can be achieved by simply dividing the data by standard deviation of the data. No per-channel normalisation is required. To preserve the units and original data scale, store the original data variance and multiply the _jDAS_ reconstructions after the denoising step:

```
scale = data.std()  # Store original data scale
new_data = ...      # jDAS denoising step
new_data *= scale   # Restore data scale
```

The data in this example are already normalised.

In [4]:
samp = 50.    # Time-sampling frequency [Hz]
gauge = 19.2  # Gauge length [m]

# Load data
data = np.load("sample_data.npy")
Nch, Nt = data.shape

# Define channel positions and time vector
dist = np.arange(Nch) * gauge * 1e-3
t = np.arange(Nt) / samp

# Denoise data

Since the model has been trained on data from the same DAS experiment as the data sample loaded above, retraining the model is not necessary. In a more realistic scenario, when the model is applied to new data, some retraining could improve the denoising accuracy. See `retraining_example.ipynb` for an example of how to retrain the model.

Denoising data with _jDAS_ involves 3 simple steps:

1. Instantiate the `JDAS` class.
2. Load a pretrained model with `JDAS.load_model()`. In this example we load a standard pretrained model, so we set `model_path=None` (default). In the case a model is available that has been retrained on a specific data set, it can be loaded by pointing `model_path` to the directory where the frozen model is stored (e.g. `model_path="path/to/model/saved-model.h5"`).
3. Call the denoising routine `JDAS.denoise()` with the data to be denoised. Under the hood, this routine breaks up the data into small chunks of 2048 time samples and reconstructs the data channel-by-channel, after which the junks are concatenated to obtain the same shape as the input (minus the remainder of dividing the data into an integer number of chunks). In some cases it could happen that parasitic low-frequency artifacts are introduced, which can be filtered out with a standard bandpass filter. Since the input data was filtered in a 1-10 Hz pass band, we will apply the same pass band to the _jDAS_ reconstructed data. This behaviour can be disabled by setting `postfilter=False` (default). 

Note that when calling `JDAS.denoise` for the first time in a session, Keras/TensorFlow will rebuild and optimise the model, which takes some time (of the order of 10 seconds). Once this initiation step is done, the subsequent calls to the model are very fast.

In [5]:
fmin = 1.
fmax = 10.

jdas = JDAS()
model = jdas.load_model()
clean_data = jdas.denoise(data, postfilter=True, filter_band=(fmin, fmax, samp))

# Plot data

In [7]:
# Plotting parameters
imshow_kwargs = {
    "interpolation": "antialiased",
    "aspect": "auto",
    "vmin": -0.5,
    "vmax": 0.5,
    "extent": [t.min(), t.max(), dist.max(), dist.min()],
}

# Close all other figures
plt.close("all")
# Draw the canvas (2 columns)
fig, axes = plt.subplots(ncols=2, figsize=(9.5, 5), constrained_layout=True, sharey="all")

# Loop over columns and data sets (original and denoised data)
for ax, d, title in zip(axes, (data, clean_data), ("Original DAS data", "$jDAS$ reconstruction")):
    ax.imshow(d, **imshow_kwargs)
    ax.set_xlabel("Time [s]")
    ax.set_title(title)
    
axes[0].set_ylabel("Distance along cable [km]")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …